In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pytorch_lightning

     |████████████████████████████████| 915 kB 11.2 MB/s 
     |████████████████████████████████| 10.6 MB 23.2 MB/s 
     |████████████████████████████████| 234 kB 78.9 MB/s 
     |████████████████████████████████| 636 kB 53.9 MB/s 
     |████████████████████████████████| 118 kB 61.0 MB/s 
     |████████████████████████████████| 829 kB 52.3 MB/s 
     |████████████████████████████████| 1.3 MB 54.4 MB/s 
     |████████████████████████████████| 142 kB 64.1 MB/s 
     |████████████████████████████████| 294 kB 64.6 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=d041f78cda2824da987e2151da216ce3e11d2fb7b846a07a08a078323a4eca05
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.5.0
    Uninstalling tensorboard-2.5.0:
      Successfully uninstalled tensorboard-2.5.0
  Attempting uninsta

# Preparation

## Import Packages

In [ ]:
import torch 
import torch.nn as nn
import pytorch_lightning as pl
import torch.nn.functional as F
from torch import bernoulli, tensor, randint, ones, cat, cuda
import pandas as pd
import torch.optim as optim
import numpy as np
import csv, json, os, random
from torch.utils.data import Dataset, DataLoader, random_split
from tqdm import tqdm

## Initialize File Readers


In [ ]:
def read_csv(file_dir):

  lst = []
  with open(file_dir, 'r') as f:
    reader = csv.reader(f)
    i=0
    for row in reader:
      fact = []
      for i in range(len(row)):
        if i == 3:
          fact.append(float(row[i]))
        else:
          fact.append(int(row[i]))
      lst.append(fact)

  return lst

def read_dict(file_dir):
  d = {}
  with open(file_dir, 'r') as f:
    reader = csv.reader(f)
    for k, v in reader:
     
      d[k] = v

  return d


# Modules

## TransD Model 


In [ ]:

class TransD(pl.LightningModule):
  """
  Paramters: 
    self.dim_e: dimension of entity embedding 
    self.dim_r: dimension of relation embedding
    self.ent_total: number of entities in knowledge graph
    self.rel_total: number of relations in knowledge graph 
    max_norm: max of L2 norm for embedding
    gamma: margin hyperparameter for loss function 
    beta: hyperparameter for focusE scoring function, beta=0 indicates the use of 
      number edge weightings, beta=1 indicates conventional KGE architecture 
  
  Attributes: 
    ent_embeddings: vector embeddings for entities 
    rel_embeddings: vector embeddings for relations 
    ent_proj_embeddings: vector embeddings for entity projections 
    rel_proj_embeddings: vector embeddings for relation projections 


  """
  def __init__(self, dim_r, dim_e, ent_total, rel_total, max_norm, gamma, beta, 
               sampler, num_classes, lr, train_dl, val_dl):
    super(TransD, self).__init__()

    # Initialize model parameters
    self.dim_e = dim_e
    self.dim_r = dim_r
    self.ent_total = ent_total 
    self.rel_total = rel_total 
    self.max_norm = max_norm
    self.gamma = gamma
    self.beta = beta
    self.sampler = sampler
    self.num_classes = num_classes
    self.lr = lr
    self.train_dl = train_dl
    self.val_dl = val_dl


    # Create embeddings
    self.ent_embeddings = nn.Embedding(self.ent_total+1, self.dim_e, 
                                       max_norm=self.max_norm)
    self.rel_embeddings = nn.Embedding(self.rel_total+1, self.dim_r,  
                                       max_norm=self.max_norm)
    self.ent_proj_embeddings = nn.Embedding(self.ent_total + 1, self.dim_e, 
                                       max_norm=self.max_norm)
    self.rel_proj_embeddings = nn.Embedding(self.rel_total + 1, self.dim_r,  
                                       max_norm=self.max_norm)
    
    
  def forward(self, h, t, r, w): 
    """
    Parameters:
      h: torch.Tensor, shape: (N x 1)
        Head entities in current batch
      t: torch.Tensor, shape: (N x 1)
        Tail entities in current batch 
      r: torch.Tensor, shape: (N x 1)
        Relations in current batch 
      w: torch.Tensor, shape: (N x 1)
        Relation weights in current batch 
    Returns: 
      projected_h: torch.Tensor, shape: (N x dim_r x 1)
        Head projection vector
      projected_t: torch.Tensor, shape: (N x dim_r x 1)
        Tail projection vector
      r_e: torch.Tensor, shape: (N x dim_r)
        Relation vector embedding 
      w: torch.Tensor, shape: (N x 1)
        Relation weights in current batch
    """
   

    h_e = self.ent_embeddings(h) # N x dim_e
    t_e = self.ent_embeddings(t) # N x dim_e
    r_e = self.rel_embeddings(r) # N x dim_e
    h_proj_e = self.ent_proj_embeddings(h) # N x dim_e
    t_proj_e = self.ent_proj_embeddings(t) # N x dim_e
    r_proj_e = self.rel_proj_embeddings(r) # N x dim_r
    

    
    M_rh = self.projection(r_proj_e, h_proj_e) # N x dim_r x dim_e
    M_rt = self.projection(r_proj_e, t_proj_e) # N x dim_r x dim_e

    # proj_h =
    projected_h = torch.matmul(M_rh, h_e.view(-1, self.dim_e, 1)) # N x dim_r x 1
    projected_t = torch.matmul(M_rt, t_e.view(-1, self.dim_e, 1)) # N x dim_r x 1

    return projected_h.view(-1, self.dim_r), projected_t.view(-1,self.dim_r), r_e, w
  
  def score(self, projected_h, projected_t, r_e, weight, pos):
    """
    Parameters: 
      projected_h: torch.Tensor, shape: (N x dim_r x 1)
        Head projection vector
      projected_t: torch.Tensor, shape: (N x dim_r x 1)
        Tail projection vector
      r_e: torch.Tensor, shape: (N x dim_r)
        Relation vector embedding 
      w: torch.Tensor, shape: (N x 1)
        Relation weights in current batch
    Returns: 
      score: float, shape: (N) 
        Scoring function 
    """
    
    transD_score = -1*(projected_h + r_e - projected_t).square().sum(dim=1)
    
    if pos:
      alpha = self.beta + (1-weight)*(1-self.beta) 
    else:
      alpha = self.beta + weight*(1-self.beta)

    m = nn.Softplus()
    g = m(transD_score)

    score = alpha*g
    
    return score
    

  def projection(self,r_proj_e, e_proj):
    """
    Parameters:
    r_proj_e: torch.Tensor, shape: (N x dim_r)
      Relation projection vector embedding 
    e_proj: torch.Tensor, shape (N x dim_e) 
      Entity projection vector embedding (N x dim_e)
    Returns:
      matrix_map: torch.Tensor, shape(N x dim_r x dim_e) 
        Mapping matrix that projects entity to relation space 
    """
    N, dim_r = r_proj_e.shape
    dim_e = e_proj.shape[1] 
    
    matrix_map = r_proj_e.view(N,dim_r,1).matmul(e_proj.view(N,1,dim_e)) + torch.eye(dim_r, dim_e, device=e_proj.device)
    
    return matrix_map

  
  def loss(self, pos, neg):
    """
    Parameters: 
      neg: torch.Tensor, shape: (N)
        Score of negative triplets 
      pos: torch.Tensor, shape (N)
        Score of golden triplets
      gamma: float [0,1], dtype: float 
        Margin separating golden and negative triplets 
    Returns:
      loss: dtype: float
        Margin-based ranking loss 
    """
    loss = (pos.view(-1,1) + neg.view(1,-1)+ self.gamma).relu().sum()

    return loss/(pos.shape[0]*neg.shape[0])
  
  def configure_optimizers(self):
    optimizer = torch.optim.AdamW(self.parameters(), lr=(self.lr)) 
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)
    # cosine  
    return [optimizer], [scheduler]

  def training_step(self, train_batch, batch_idx):
    h, t, r, w = train_batch[0], train_batch[1], train_batch[2], train_batch[3]
    

    neg_h, neg_t = self.sampler.corrupt_batch(h,t,r)
    
    # Push tensors to GPU
   
    # p_proj_h, p_proj_t, p_r_e, w = self.forward(h.cuda(),t.cuda(),r.cuda(), w.cuda())
    p_proj_h, p_proj_t, p_r_e, w = self.forward(h,t,r, w)
    n_proj_h, n_proj_t, n_r_e, w = self.forward(neg_h, neg_t, r, w)

    pos_score = self.score(p_proj_h, p_proj_t, p_r_e, w, True)
    neg_score = self.score(n_proj_h, n_proj_t, n_r_e, w, False)
    loss = self.loss(pos_score, neg_score)
    self.log('train_loss', loss)

    return loss

  def validation_step(self, val_batch, batch_idx):

    h, t, r, w = val_batch[0], val_batch[1], val_batch[2], val_batch[3]

    
    neg_h, neg_t = self.sampler.corrupt_batch(h,t,r)
    
    p_proj_h, p_proj_t, p_r_e, w = self.forward(h,t,r, w)
    n_proj_h, n_proj_t, n_r_e, w = self.forward(neg_h, neg_t, r, w)
    # p_proj_h, p_proj_t, p_r_e, w = self.forward(h.cuda(),t.cuda(),r.cuda(), w.cuda())
    # n_proj_h, n_proj_t, n_r_e, w = self.forward(neg_h.cuda(), neg_t.cuda(), r.cuda(), w.cuda())

    pos_score = self.score(p_proj_h, p_proj_t, p_r_e, w, True)
    neg_score = self.score(n_proj_h, n_proj_t, n_r_e, w, False)
    loss = self.loss(pos_score, neg_score)

    self.log('val_loss', loss)
  def train_dataloader(self):

    return self.train_dl

  def val_dataloader(self):

    return self.val_dl



## Bernoulli Sampler


In [ ]:
class BernoulliNegativeSampler():
  def __init__(self, kg, n_neg, batch_size, rel_total, ent_total):
    super(BernoulliNegativeSampler, self).__init__() 

    self.kg = kg
    self.n_neg = n_neg 
    self.batch_size = batch_size
    self.rel_total = rel_total
    self.ent_total = ent_total
    #self.bern_probs = self.get_bern_probs(kg)

  def corrupt_batch(self, heads, tails, relations, n_neg=1):
    """
    Parameters:  
      heads: tensor containing the integer key of heads of the realtions in current
        batch (N)
      tails: tensor containing the integer key of tails of the relations in current
        batch (N)
      relations: tensor containing the integer key of relations in the current batch. (N)
      n_neg - number of negative samples to create from each fact 
    Returns: 
      neg_heads: tensor containing the integer key of negatively sampled heads of 
        the realtions in the current batch. (N)
      neg_tails: tensor containing the integer key of negatively sampled tails of 
        the realations in the current batch. (N) 
    """
    
    device = heads.device

    neg_heads = heads.repeat(n_neg) # N x n_neg
    neg_tails = tails.repeat(n_neg) # N x n_neg
    facts = torch.cat((heads.view(-1,1), tails.view(-1,1), relations.view(-1, 1)), dim=1)
    #print(facts.shape)
    bern_probs = self.evaluate_probabilites(facts)
    #print(bern_probs.keys())

    mask = bernoulli(bern_probs[relations].repeat(n_neg)).double()
    n_h_cor = int(mask.sum().item())
    # if mask == 1, corupt heads
    neg_heads[mask==1] = randint(1, self.ent_total, (n_h_cor,), device=device)
    # if mask == 0, corupt tails
    neg_tails[mask==0] = randint(1, self.ent_total, (heads.shape[0] * n_neg - n_h_cor,), device=device)

    return neg_heads.long(), neg_tails.long() 


  def evaluate_probabilites(self, facts):
    """
    Parameters:
      kg: torch.Tensor, shape (N x 3)
        All knowledge graph triples 
    Returns: 
      tensor with relation probabilities 
    """
    bern_probs = self.get_bern_probs(facts)

    temp = [] 
    for i in range(self.rel_total):
      if i in bern_probs.keys():
        temp.append(bern_probs[i])
      else:
        temp.append(0.5)

    return tensor(temp).float()

  def get_hpt(self, t):
    """
    Parameters: 
      t: torch.Tensor, shape (N x3) 
        Tensor of encoded triples in current batch (N x 3), first column contains 
        head indicies, second tails and third relations
    Returns: 
      d: dictionary of relation indices to average number of heads per tail
    """
    t = t.cpu()
    df_hpt = pd.DataFrame(t.numpy(), columns=['from', 'to', 'rel'])
    df_hpt = df_hpt.groupby(['rel', 'to']).count().groupby('rel').mean()
    df_hpt.reset_index(inplace=True)
    
    return {df_hpt.loc[i].values[0]: df_hpt.loc[i].values[1] for i in df_hpt.index}
  
  def get_tph(self, t):
    """
    Parameters: 
      t: torch.Tensor, shape (N x 3) 
        Tensor of encoded triples in current batch (N x 3), first 
        column contains head indicies, second tails and third relations
    Returns: 
      d: dictionary of relation indices to average number of tails per head
    """
    t = t.cpu()
    df_tph = pd.DataFrame(t.numpy(), columns=['from', 'to', 'rel'])
    df_tph = df_tph.groupby(['from', 'rel']).count().groupby('rel').mean()
    df_tph.reset_index(inplace=True)

    return {df_tph.loc[i].values[0]: df_tph.loc[i].values[1] for i in df_tph.index}
   
  def get_bern_probs(self, t):
    """
    Parameters: 
      t: torch.Tensor, shape: (Nx3)
        Tensor of encoded triples in current batch (Nx3), first 
        column contains head indicies, second tails and third relations
    Returns: 
      tph: dictionary that maps relations to sampling probabilities 
    """

    hpt = self.get_hpt(t)
    tph = self.get_tph(t) 

    assert (tph.keys() == hpt.keys())

    
    for k in tph.keys():
      tph[k] = tph[k] / (tph[k] + hpt[k])

    return tph



## Knowledge Graph Class

In [ ]:
USE_CUDA = torch.cuda.is_available()
if USE_CUDA:
	longTensor = torch.cuda.LongTensor
	floatTensor = torch.cuda.FloatTensor


else:
	longTensor = torch.LongTensor
	floatTensor = torch.FloatTensor

class KnowledgeGraph(Dataset):
  """
  Parameters: 
    facts: 2D array, dtype: int, shape: (n_facts x 3)
      All fact triplets in knowledge graph, where facts[0] is the head, facts[1] 
      is the tail and facts[2] is the relation
    ent2id: dict, dtype: {str: int}, shape: (ent_total)
      Dictionary mapping entity label to integer index 
    rel2id: dict, dtype {str:int}, shape: (rel_total)
      Dictionary mapping relation label to integer index 
    id2ent: dict, dtype: {int: str}, shape: (ent_total)
      Dictionary integer index to entity label
    id2rel: dict, dtype: {int: str}, shape: (ent_total)
      Dictionary integer index to relation label 
    Attributes: 
      n_facts: int 
        Number of facts in data set 
      ent_total: int 
        Number of unique entities in data set 
      rel_total: int 
        Number of unique relations in data set 
      head_index: torch.Tensor, dtype: torch.long, shape (n_facts)
        List of int index of heads for facts, where the ith element represents the
        head of ith triplet
      tail_index: torch.Tensor, dtype: torch.long, shape (n_facts)
        List of int index of tail for facts, where the ith element represents the
        tail of ith triplet
      tail_index: torch.Tensor, dtype: torch.long, shape (n_facts)
        List of int index of relations for facts, where the ith element represents the
        tail of ith triplet 
      weight_index: torch.Tensor, dtype: torch.float, shape (n_facts)
        List of weights for facts, where the ith element respresents the numeric
        edge weight of the ith triplet
  """
  def __init__(self, facts, id2ent, ent2id, id2rel, rel2id):
    super(KnowledgeGraph, self).__init__()
    # Initialize
    self.kg = facts
    self.ent2id = ent2id
    self.id2ent = id2ent
    self.rel2id = rel2id
    self.id2rel = id2rel 

    # Set Attributes
    self.n_facts = len(facts)
    self.ent_total = len(id2ent.keys())
    self.rel_total = len(id2rel.keys())
    
    head_index, tail_index, rel_index, weight_index = [], [], [], []
    for fact in facts[1:]:
      if fact[0] != 's':
        head_index.append(int(fact[0]))
        tail_index.append(int(fact[1]))
        rel_index.append(int(fact[2]))
        weight_index.append(float(fact[3]))
    
    self.head_index = tensor(head_index).long()#.cuda()
    self.tail_index = tensor(tail_index).long()#.cuda()
    self.rel_index = tensor(rel_index).long()#.cuda()
    self.weight_index = tensor(weight_index).float()#.cuda()



  def __len__(self):
    return self.n_facts -1

  def __getitem__(self, item):
    """
    Parameters: 
      item: 
    """
    return (self.head_index[item].item(), 
            self.tail_index[item].item(), 
            self.rel_index[item].item(), 
            self.weight_index[item].item())
    
  def get_ent_id(self, ent):
    """
    Parameters: 
      ent: torch.Tensor, dtype: str, shape: (Nx1)
        Enity string in knowledge graph
    Returns: 
      id: torch.Tensor, dtype: int, shape: (Nx1)
        Integer id of entity, returns None if entity does not exist
    """
    if ent in self.ent2id:
      id = self.ent2id[ent]
    else:
      print("KeyError: Entity does not exist")
      id = None
    return id
    
  def get_rel_id(self, rel):
    """
    Parameters: 
      rel: torch.Tensor, dtype: str, shape: (Nx1)
        Relation string in knowledge graph
    Returns: 
      id: torch.Tensor, dtype: int, shape: (Nx1)
        Integer id of relation, returns None if relation does not exist
    """
    if rel in self.rel2id:
      id = self.rel2id[ent]
    else:
      print("KeyError: Relation does not exist")
      id = None
    return id

  def get_id_rel(self, id):
    """
    Parameters: 
      id: torch.Tensor, dtype: str, shape: (Nx1)
        Integer id of relation
    Returns: 
      rel: torch.Tensor, dtype: int, shape: (Nx1)
        Relation string in knowledge graph, returns None if id does not exist
    """

    if id in self.id2rel:
      rel = self.id2rel[id]
    else:
      print("KeyError: Relation ID does not exist.")
    return rel

  def get_id_ent(self, id):
    """
    Parameters: 
      id: torch.Tensor, dtype: str, shape: (Nx1)
        Integer id of relation
    Returns: 
      ent: torch.Tensor, dtype: int, shape: (Nx1)
        Entity string in knowledge graph, returns None if id does not exist
    """
    if id in self.id2ent:
      ent = self.id2ent[id]
    else:
      print("KeyError: Entity ID does not exist.")
      ent = None
    return ent
    
  def heads(self):
    """
    Returns: 
      heads: torch.Tensor, dtype: torch.int, shape (N x 1)
        Integer id's of heads in current batch 
    """
    kg = self.kg 
    heads = []
    for trip in kg:
      heads.append(trip[0])
    return tensor(heads).long()

  def tails(self):
    """
    Returns: 
      tails: torch.Tensor, dtype: torch.int, shape (N x 1)
        Integer id's of tails in current batch 
    """
    kg = self.kg 
    tails = []
    for trip in kg:
      tails.append(trip[3])
    return tensor(tails).long()



## Data Module

In [ ]:
class TransDDataModule(pl.LightningDataModule):
  def __init__(self, batch_size, kg_dir, id2ent_dir, id2rel_dir, ent2id_dir, rel2id_dir):
    super().__init__()
    
  #def setup(self, stage=None):
    
    self.batch_size = batch_size
    self.kg_dir = kg_dir
    self.id2rel_dir = id2rel_dir
    self.ent2id_dir = ent2id_dir
    self.id2ent_dir = id2ent_dir
    self.rel2id_dir = rel2id_dir

    triplets = read_csv(self.kg_dir)[1:]
    id2ent = read_dict(self.id2ent_dir)
    ent2id = read_dict(self.ent2id_dir)
    id2rel = read_dict(self.id2rel_dir)
    rel2id = read_dict(self.rel2id_dir) 

    self.facts = triplets
    self.kg = KnowledgeGraph(self.facts, id2ent, ent2id, id2rel, rel2id)
    
    train_size = round(len(self.facts)*0.9) - round(len(self.facts)*0.9)%1024
    val_size = len(self.facts) - train_size -1

    self.kg_train, self.kg_val = random_split(self.kg, [train_size, val_size]) 
  #def setup(self, stage=None):

  def train_dataloader(self):
    return DataLoader(self.kg_train, batch_size=self.batch_size, shuffle=True, num_workers=4)

  def val_dataloader(self):
    return DataLoader(self.kg_val, batch_size=self.batch_size, num_workers=4)



# Learn

In [ ]:
batch_size = 256
kg_dir = '/content/drive/MyDrive/UCLA REU 2021 KG /results/2020Mar11-2021June16/tripples/all_relations-peaks-v2.csv'
id2ent_dir = '/content/drive/MyDrive/UCLA REU 2021 KG /results/2020Mar11-2021June16/tripples/id2entity-peaks.csv'
ent2id_dir = '/content/drive/MyDrive/UCLA REU 2021 KG /results/2020Mar11-2021June16/tripples/entity2id-peaks.csv'
id2rel_dir = '/content/drive/MyDrive/UCLA REU 2021 KG /results/2020Mar11-2021June16/tripples/id2relation-peaks.csv'
rel2id_dir = '/content/drive/MyDrive/UCLA REU 2021 KG /results/2020Mar11-2021June16/tripples/relation2id-peaks.csv'

triplets = read_csv(kg_dir)[1:]
id2ent = read_dict(id2ent_dir)
ent2id = read_dict(ent2id_dir)
id2rel = read_dict(id2rel_dir)
rel2id = read_dict(rel2id_dir) 


kg = KnowledgeGraph(triplets, id2ent, ent2id, id2rel, rel2id)

train_size = round(len(triplets)*0.98) - round(len(triplets)*0.98)%256 
val_size = len(triplets) - train_size -1

kg_train, kg_val = random_split(kg, [train_size, val_size]) 

In [ ]:
print(max([x[0] for x in triplets]))
print(len(id2ent.keys()))
print(len(ent2id.keys()))

711782
711783
711783


In [ ]:

train_batch = DataLoader(kg_train, batch_size=batch_size, shuffle=True, num_workers=4)
val_batch = DataLoader(kg_val, batch_size=batch_size, num_workers=4)

sampler = BernoulliNegativeSampler(kg=kg, 
                                   n_neg=1, 
                                   batch_size=batch_size, 
                                   rel_total=kg.rel_total, 
                                   ent_total=kg.ent_total)

model = TransD(dim_r=100, 
               dim_e=100, 
               ent_total=kg.ent_total, 
               rel_total=kg.rel_total, 
               max_norm=1, 
               gamma=0.5, 
               beta=0.9, 
               sampler=sampler,
               num_classes=2,
               lr=1e-3, 
               train_dl = train_batch,
               val_dl = val_batch).cuda()


checkpoint_callback = pl.callbacks.ModelCheckpoint(dirpath='/content/drive/MyDrive/UCLA REU 2021 KG /KG Embedding')

trainer = pl.Trainer(gpus=1,
                     default_root_dir='/content/drive/MyDrive/UCLA REU 2021 KG /KG Embedding',
                     max_epochs=15, 
                     auto_lr_find=True,
                     callbacks = [checkpoint_callback])


trainer.tune(model)

trainer.fit(model, train_batch, val_batch)
# resume_from_checkpoint = ‘’

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:446: UserWarning: Checkpoint directory /content/drive/MyDrive/UCLA REU 2021 KG /KG Embedding exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                | Type      | Params
--------------------------------------------------
0 | ent_embeddings      | Embedding | 71.2 M
1 | rel_embeddings      | Embedding | 1.2 K 
2 | ent_proj_embeddings | Embedding | 71.2 M
3 | rel_proj_embeddings | Embedding | 1.2 K 
--------------------------------------------------
142 M     Trainable params
0         Non-trainable params
142 M     Total params
569.437   Total estimated model params size (MB)


Restoring states from the checkpoint file at /content/drive/MyDrive/UCLA REU 2021 KG /KG Embedding/lr_find_temp_model.ckpt
Restored all states from the checkpoint file at /content/drive/MyDrive/UCLA REU 2021 KG /KG Embedding/lr_find_temp_model.ckpt
Learning rate set to 0.09120108393559097
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                | Type      | Params
--------------------------------------------------
0 | ent_embeddings      | Embedding | 71.2 M
1 | rel_embeddings      | Embedding | 1.2 K 
2 | ent_proj_embeddings | Embedding | 71.2 M
3 | rel_proj_embeddings | Embedding | 1.2 K 
--------------------------------------------------
142 M     Trainable params
0         Non-trainable params
142 M     Total params
569.437   Total estimated model params size (MB)
